1. 네이버 자료

2. 네이버 Place 코드
    * 기준: 서울시 동별, 업종별
    
    
3. 가게정보, 리뷰 정도 겟



# 필요함수 및 라이브러리

In [1]:
import time
import pandas as pd
import numpy as np
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


## 파일 관련

In [2]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [3]:
def save_df(df, path, file_name):
    df.to_csv(path+file_name, index=False)
    print(f'{path+file_name} 저장 완료')

In [4]:
def read_csv_코드정보(code_path , code_file):
    df = pd.read_csv(code_path + code_file )
    return df

###### 서울시 동 load 관련

In [5]:
import pickle
import collections

def get_seoul_dong():
    with open('data/서울시_구별_동정리.pickle','rb') as fw:
        dong_dict = pickle.load( fw)

    dong_key_list = list(dong_dict.keys())
    dong_key_list.sort()
    # dong_key_list

    for 구이름 in dong_key_list:
        dong_dict[구이름] = sorted(dong_dict[구이름])
    
    return dong_dict

In [6]:
dong_dict = get_seoul_dong()

## 웹 조정 관련

In [7]:
def is_clickable(wd):
    return wd.is_enabled() and wd.is_displayed()

In [8]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## m.map.naver.com 

### 자기 위치 설정 함수

In [9]:
def set_location(wd, map_url, 구이름, 동이름, detail=False, slow=1):
    
    if detail:
        print('#####'+동이름+'____'+구이름+'#####')
        
    wd.get(map_url)
    time.sleep(2*slow)

    #### 위치 지정
    wd.find_element_by_id('_changeLocation').click()

    try:
        wd.find_element_by_class_name('mylct_rm_mloc').click()
    except NoSuchElementException:
        if detail:
            print('S_S_E')
        pass
    
    time.sleep(0.2*slow)
    wd.find_element_by_class_name('mylct_sch_query').click()
    time.sleep(0.2*slow)
    wd.find_element_by_class_name('mylct_sch_query').send_keys(구이름+' '+동이름)
    time.sleep(0.2*slow)
    wd.find_element_by_class_name('mylct_spr.mylct_ico_sch').click()
    time.sleep(0.2*slow)
    wd.find_element_by_class_name('mylct_addr.mylct_sch_rst').click()
    time.sleep(1*slow)
    
    #### 업종 지정
    category_1 = wd.find_elements_by_class_name('_linkCategory')
    # 업종 선택 : 선택 가능 
    '''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
       주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
       영화공연:CINEMA, 관공서:GOVERNMENT'''
    
    for bnt_ in category_1:
        if is_clickable(bnt_):
            if bnt_.get_attribute('data-category') == 업종:
                bnt_.click()
                time.sleep(0.2*slow)
                break
    time.sleep(3*slow)

    

In [10]:
    '''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
       주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
       영화공연:CINEMA, 관공서:GOVERNMENT'''

'음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,\n   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, \n   영화공연:CINEMA, 관공서:GOVERNMENT'

### 코드 수집 함수

In [11]:
def get_store_code(구이름, 업종, detail=False, slow=1):
    
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1)

    map_url = 'https://m.map.naver.com/search2/interestSpot.naver?type=DINING'

    df = pd.DataFrame(columns=('store_name','store_code'))
    code_idx=0
    동_idx=0
    start_time_all = time.time()
    print(f'{구이름} 코드 수집을 시작합니다.')
    
    for 동이름 in dong_dict[구이름]:
        print(f'{동이름} 시작')
        start_time= time.time()
        # 각각의 현재 위치를 조정
        set_location(wd, map_url, 구이름, 동이름, detail=detail, slow=1)

        ### 소분류 url 생성
        current_url = wd.current_url.split('type=')[0]
        category_2_url_list=[]
        category_2 = wd.find_elements_by_class_name('_linkSubCategory')
        for 소분류 in category_2:
            category_2_url_list.append(current_url+'type='+소분류.get_attribute('data-category_call_id'))
        scroll_down(wd,1*slow)


        ### 이름 및 코드 수집
        for url_ in category_2_url_list:
            wd.get(url_)
            time.sleep(3*slow)
            scroll_down(wd,1*slow)
            store_list = wd.find_elements_by_class_name('_item._lazyImgContainer')
            
            for cd in store_list:
                store_name = cd.get_attribute('data-title')
                store_code = cd.get_attribute('data-sid')
                df.loc[code_idx] = store_name ,store_code
                code_idx +=1
                print('#',end='')
        print(f'{동이름} 완료, {time.time()-start_time} 소요')
        동_idx += 1
        
        if (동_idx+1)%5 == 0: # 쿨링 다운~
            wd.close()
            time.sleep(5*slow)
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            time.sleep(5*slow)
            print('reboot chrome')
        
    df = df.drop_duplicates()
    wd.close()
    print(f'{구이름} {df.shape[0]}개 수집 완료, {time.time()-start_time_all} 소요')
    return df

# 셋팅

In [12]:
# code_path = 'data/코드/'
# review_path = 'data/리뷰/'
# store_path = 'data/가게/'

# code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'

In [13]:
code_path = 'data/코드/'
# code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'

In [14]:
detail= True
slow=1

#  코드 수집 및 저장

In [ ]:
업종= 'DINING'

for 구이름 in sorted(dong_dict.keys())[::-1]:
    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    if check_file(code_path, code_file):
        print('@@@@@@@@@@',구이름, '완료','@@@@@@@@@@@@')
        
        pass
    else:
        df = get_store_code(구이름, 업종, detail=detail, slow=slow)
        save_df(df, code_path, code_file)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\N\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


중랑구 코드 수집을 시작합니다.
망우본동 시작
#####망우본동____중랑구#####
S_S_E
################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################망우본동 완료, 73.45836925506592 소요
망우제3동 시작
#####망우제3동____중랑구#####
###########################################################################

# 각 구 통합

In [ ]:
code_path = 'data/코드/'
code_file_list = ['가게코드_' + 구이름 +  '_' + 업종 + '.csv' for  동이름 in dong_dict[구이름]]
result_file_name = '가게코드_서울시_'+ 업종+'.csv'
df_result= pd.DataFrame()

for 구이름 in sorted(dong_dict.keys()):
    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    if check_file(code_path, code_file):
        print('구이름', '완료')
        pass
    else:
        print('구이름', '없습니다.')
        break
else:
    for code_file in code_file_list:
        df_1 = read_csv_코드정보(code_path, code_file)
        df_result = pd.concat([df_result, df_1])
    else:
        if check_file(code_path, result_file_name): # 기존것과 병합
            df_before = read_csv_코드정보(code_path, result_file_name)
        else:
            df_before = pd.DataFrame()
    df_result = pd.concat([df_before, df_result ])
    df_result = df_result.drop_duplicates()
    save_df(df_result, code_path, result_file_name )